In [83]:
import pandas as pd

In [126]:
# 1872-11-30 至今的国际比赛数据
result = pd.read_csv(r'results.csv')
martchs_set = result[result['date'] > '2008-01-01']
result.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [127]:
match_schedule = pd.read_csv(r'fifa-world-cup-2018-RussianStandardTime.csv')
group_teams = match_schedule.dropna(subset=['Group'])
world_cup_teams = pd.concat((group_teams['Home Team'],group_teams['Away Team'])).drop_duplicates()
team_list = world_cup_teams.reset_index(drop=True).tolist()

In [128]:
sr_df = pd.read_excel(r'world_cup2018_social_and_world_rank.xlsx')
for c in ['win_cnt_10year', 'match_cnt_10year', 'ratio_10year']:
    sr_df[c] = None

In [129]:
his_result = martchs_set[martchs_set['home_team'].isin(world_cup_teams) & martchs_set['away_team'].isin(world_cup_teams)]
his_result = his_result.reset_index(drop=True)

wd_tm_status = {}
for x in team_list:
#     测试球队是否都存在与名单中
    test_team_str = his_result[(his_result['home_team'] == x) | ((his_result['away_team'] == x))]
    if len(test_team_str.index) == 0:
        print(x, 'team not find his_match')
        break
    all_diff = []
    
#     统计球队的胜率及场次
    home_team_result = his_result[(his_result['home_team'] == x)]
    home_diff = (home_team_result['home_score'] - home_team_result['away_score']).tolist()
    all_diff.extend(home_diff)
    away_team_result = his_result[(his_result['away_team'] == x)]
    away_diff = (home_team_result['away_score'] - home_team_result['home_score']).tolist()
    all_diff.extend(away_diff)

    index = sr_df[sr_df['country_en'] == x].index
    sr_df.loc[index,'win_cnt_10year'] = len([x for x in all_diff if x > 0])
    sr_df.loc[index,'match_cnt_10year'] = len(all_diff)

sr_df['ratio_10year'] = round(sr_df['win_cnt_10year']/sr_df['match_cnt_10year'],3)

In [131]:
sr_df = sr_df.sort_values(by='world_rank').reset_index(drop=True)
sr_df.head()

,country_en,country_ch,world_rank,social status(100mGBP),win_cnt_10year,match_cnt_10year,ratio_10year
0,Germany,德国,1,8.72,27,66,0.409
1,Brazil,巴西,2,9.50,28,66,0.424
2,Belgium,比利时,3,7.58,16,40,0.400
3,Portugal,葡萄牙,4,4.68,18,50,0.360
4,Argentina,阿根廷,5,7.08,29,72,0.403


In [134]:
sr_df.to_excel('sr_df.xlsx')

-0.4999999999999999